In [ ]:
# Colab에서 Google 드라이브를 마운트하는 코드

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
!ls

In [ ]:
from minone import CD, model, trainer, tokenization

In [ ]:
import math
import random
import os
import numpy as np
import pandas as pd
import re

from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.nn as nn

from transformers import AutoTokenizer, AutoModel
from transformers import DistilBertModel

from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [ ]:
CFG = {
    'EPOCHS': 10,
    'LEARNING_RATE': 1e-4,
    'BATCH_SIZE': 64,
    'SEED': 4
}

In [ ]:
def seed_everything(seed=CFG['SEED']):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
text_data = pd.read_csv('../data/train_data.csv')

In [ ]:
models = DistilBertModel.from_pretrained('monologg/distilkobert')
tokenizers = tokenization.KoBertTokenizer.from_pretrained('monologg/kobert')

In [ ]:
# 모델 생성
c_model = model.CustomModel(models, 768, 35)

# 미리 학습된 가중치 파일 경로
model_checkpoint = "./code/minone/model.pth"

# 모델 가중치 파일이 존재하는 경우에만 로드
if os.path.exists(model_checkpoint):
    parameter = torch.load(model_checkpoint)
    model_keys = set(c_model.state_dict().keys())
    pretrained_keys = set(parameter.keys())
    unexpected_keys = pretrained_keys - model_keys

    trimmed_state_dict = {k: v for k, v in parameter.items() if k not in unexpected_keys}

    c_model.load_state_dict(trimmed_state_dict, strict=False)
    print("모델 가중치를 성공적으로 로드했습니다.")
else:
    print("미리 학습된 가중치 파일이 없어 기본 모델을 사용합니다.")

In [ ]:
# test_size: 검증 세트의 비율 (예: 0.2는 80% 훈련, 20% 검증을 의미)
# random_state: 랜덤 시드 값 (동일한 결과를 재현하려면 고정된 값 사용)
# stratify: 클래스 레이블을 기반으로 계층적 분할을 수행하여 클래스 분포를 유지

train_data, val_data = train_test_split(text_data, test_size=0.2, random_state=4, stratify = text_data['topic'])

In [ ]:
# reset_index 함수를 사용하여 데이터프레임의 인덱스를 재설정

train_data = train_data.reset_index(drop = True)
val_data = val_data.reset_index(drop = True)

In [ ]:
# ClassifierDataset 클래스를 사용하여 훈련 및 검증 데이터셋을 생성
# DataLoader를 사용하여 배치로 데이터를 로드

train_dataset = CD.ClassifierDataset(train_data, tokenizers, mode="train")
valid_dataset = CD.ClassifierDataset(val_data, tokenizers, mode="train")

# 훈련 데이터로더 생성
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0, pin_memory=True)

# 검증 데이터로더 생성
val_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0, pin_memory=True)

In [ ]:
import torch.nn as nn

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # 파라미터 alpha: 클래스 불균형 보정에 사용되는 가중치
        self.gamma = gamma  # 파라미터 gamma: 손실의 강도를 조절하는 하이퍼파라미터

    def forward(self, inputs, targets):
        # 1. 교차 엔트로피 손실 계산
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')

        # 2. 확률 값을 계산하고 Focal Loss 계산
        pt = torch.exp(-ce_loss)  # 확률 값 계산 (pt = e^(-ce_loss))
        F_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss  # Focal Loss 계산

        # 3. Focal Loss의 평균 반환
        return torch.mean(F_loss)


In [ ]:
# CustomModel을 사용하여 모델을 초기화
c_model = model.CustomModel(models, 768, 35)

# AdamW를 사용하고 모델의 파라미터를 업데이트
optimizer = torch.optim.AdamW(params=c_model.parameters(), lr=CFG["LEARNING_RATE"])

# 스케줄러를 초기화합니다. 검증 손실이 개선되지 않을 경우 학습률을 조절
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

# Focal Loss를 초기화,  이 손실 함수는 모델의 출력과 실제 레이블 간의 손실을 계산
criterion = FocalLoss().to(device)

In [ ]:
model_train = trainer.Trainer(c_model, optimizer, scheduler, criterion, train_dataloader, val_dataloader,
                              CFG['EPOCHS'], device)

In [ ]:
# 모델을 훈련하고 손실, 정확도 및 F1 점수를 반환합니다.
train_l, val_l, accuracy, f1_score = model_train.train()

In [ ]:
import gc

# 현재까지 사용된 메모리를 해제합니다.
gc.collect()

# CUDA 캐시를 비워줍니다. 이것은 GPU 메모리의 일부를 비워서 메모리를 최적화합니다.
torch.cuda.empty_cache()